In [1]:
cd("/home/gridsan/aramadhan/Oceananigans.jl/")
pwd()

"/home/gridsan/aramadhan/Oceananigans.jl"

In [2]:
using Pkg
Pkg.activate(".")

"/home/gridsan/aramadhan/Oceananigans.jl/Project.toml"

In [14]:
using Oceananigans, Oceananigans.Operators

In [4]:
using Random, Statistics
using FFTW

In [5]:
Nx, Ny, Nz = 8, 8, 8
Lx, Ly, Lz = 1, 1, 1
R = rand(MersenneTwister(0), Nx, Ny, Nz)
R = R .- mean(R);

In [6]:
R2 = copy(R)
R2 .= cat(R2[:, 1:2:Ny, :], R2[:, Ny:-2:2, :]; dims=2)
R2 .= cat(R2[:, :, 1:2:Nz], R2[:, :, Nz:-2:2]; dims=3);

In [7]:
@inline W(M, k) = exp(-2im*π*k/M)

ω_4Ny_ky⁺ = reshape(W.(4Ny, 0:Ny-1), 1, Ny, 1)
ω_4Ny_ky⁻ = reshape(W.(4Ny, 0:-1:-(Ny-1)), 1, Ny, 1)
ω_4Nz_kz⁺ = reshape(W.(4Nz, 0:Nz-1), 1, 1, Nz)
ω_4Nz_kz⁻ = reshape(W.(4Nz, 0:-1:-(Nz-1)), 1, 1, Nz)

R3 = fft(R2, [2, 3])

r_inds = [1, collect(Ny:-1:2)...]
R3⁻ = view(R3, 1:Nx, r_inds, 1:Nz)

R4 = zeros(Nx, Ny, Nz)
@. R4 = 2 * real(ω_4Nz_kz⁺ * (ω_4Ny_ky⁺ * R3 + ω_4Ny_ky⁻ * R3⁻));

In [8]:
R4 ≈ FFTW.r2r(R, FFTW.REDFT10, [2, 3])

true

In [9]:
R4 = fft(R4, 1);

In [25]:
is = reshape(1:Nx, Nx, 1, 1)
js = reshape(1:Nx, 1, Ny, 1)
ks = reshape(1:Nx, 1, 1, Nz)

kx² = @. (2sin((is-1)*π/Nx)    / (Lx/Nx))^2
ky² = @. (2sin((js-1)*π/Ny)    / (Ly/Ny))^2
kz² = @. (2sin((ks-1)*π/(2Nz)) / (Lz/Nz))^2

@. R4 = -R4 / (kx² + ky² + kz²)

R4[1, 1, 1] = 0;

kx²

In [11]:
R4 = ifft(R4, 1);

In [12]:
ω_4Ny_ky⁺ = reshape(W.(4Ny, 0:Ny-1), 1, Ny, 1)
ω_4Ny_ky⁻ = reshape(W.(4Ny, 0:-1:-(Ny-1)), 1, Ny, 1)
ω_4Nz_kz⁺ = reshape(W.(4Nz, 0:Nz-1), 1, 1, Nz)
ω_4Nz_kz⁻ = reshape(W.(4Nz, 0:-1:-(Nz-1)), 1, 1, Nz)

r_y_inds = [1, collect(Ny:-1:2)...]
r_z_inds = [1, collect(Nz:-1:2)...]

R4⁻⁺ = view(R4, 1:Nx, r_y_inds, 1:Nz)
R4⁺⁻ = view(R4, 1:Nx, 1:Ny, r_z_inds)
R4⁻⁻ = view(R4, 1:Nx, r_y_inds, r_z_inds)

mask_ky = ones(1, Ny, 1); mask_ky[1] = 0;
mask_kz = ones(1, 1, Nz); mask_kz[1] = 0;

R5 = zeros(Complex{Float64}, Nx, Ny, Nz)
@. R5 = 1/4 *  ω_4Ny_ky⁻ * ω_4Nz_kz⁻ * ((R4 - mask_ky * mask_kz * R4⁻⁻) - im*(mask_kz * R4⁺⁻ + mask_ky * R4⁻⁺));

In [13]:
R6 = ifft(R5, [2, 3])

R7 = zeros(Nx, Ny, Nz)

p_y_inds = [1:2:Ny..., Ny:-2:2...]
p_z_inds = [1:2:Nz..., Nz:-2:2...]
R7_p = view(R7, 1:Nx, p_y_inds, p_z_inds)

@. R7_p = real(R6)

8×8×8 view(::Array{Float64,3}, 1:8, [1, 3, 5, 7, 8, 6, 4, 2], [1, 3, 5, 7, 8, 6, 4, 2]) with eltype Float64:
[:, :, 1] =
 -0.000835371  -0.000657478   0.000613218  …   0.00175594    0.00059533 
 -0.00226279   -0.0025743    -0.000362385      0.00233606   -0.000921489
  0.000999418  -0.00202791    0.000649391      0.00462224    0.00179097 
  0.00277555   -0.000298784  -0.00112252       0.00494094    0.00297084 
  0.00363358    0.00211104    0.00010534       0.00323249    0.00350898 
  0.00406301    0.00409423    0.00328027   …   0.00249695    0.00156068 
  0.00368569    0.00225905    0.00137514      -0.000450213   0.00214412 
  0.00203141   -0.000414052  -0.000917997     -0.00123714    0.00019747 

[:, :, 2] =
  0.00156103    0.00183148    0.000649374  …  0.00074117   -0.000951664
  0.000314394  -0.000657331   0.00127987      0.00173362   -0.000978448
 -0.000873775  -0.00150707    4.13155e-5      0.0018236     0.00160255 
  0.00115314   -0.00064677   -0.00055849      0.00164709    0.0006

In [22]:
function ∇²!(grid::RegularCartesianGrid, f, ∇²f)
    for k in 1:grid.Nz, j in 1:grid.Ny, i in 1:grid.Nx
        @inbounds ∇²f[i, j, k] = ∇²(grid, f, i, j, k)
    end
end

grid = RegularCartesianGrid((Nx, Ny, Nz), (Lx, Ly, Lz))
fbcs = ChannelBCs()

ϕ   = CellField(Float64, CPU(), grid)
∇²ϕ = CellField(Float64, CPU(), grid)

data(ϕ) .= real.(R7)

fill_halo_regions!(grid, (:T, fbcs, ϕ.data))
∇²!(grid, ϕ, ∇²ϕ)

fill_halo_regions!(grid, (:T, fbcs, ∇²ϕ.data))

data(∇²ϕ) ≈ R

false

In [23]:
data(∇²ϕ)[:, :, 1]

8×8 Array{Float64,2}:
  0.307804  -0.297088   0.236128  …   0.477033  -0.41448     0.104926 
  0.425835   0.154495   0.622365      0.174897  -0.108739   -0.0289847
 -0.146143  -0.344256   0.768263     -0.617373  -0.0849669  -0.0907888
 -0.130372  -0.300511   0.663793     -0.895446   0.72848     0.0627474
 -0.128135  -0.316195   0.157008     -0.799595   0.409737   -0.515171 
 -0.30298    0.507814  -0.652372  …   0.340553  -0.0734155  -0.0115488
 -0.370852  -0.164029  -0.366813     -0.229488   0.0797792   0.0748863
 -0.279679   0.203668   0.125346      0.409012   0.178815    0.340484 

In [24]:
R[:, :, 1]

8×8 Array{Float64,2}:
  0.319189  -0.14263     0.0714293  …   0.447233  -0.320343    0.043884 
  0.405898   0.468758    0.363821       0.29666   -0.192313   -0.0275754
 -0.339892   0.0813535   0.463341      -0.380135  -0.308051    0.0979165
 -0.327129   0.0348312   0.263232      -0.390189   0.369123    0.286888 
 -0.225578  -0.244422   -0.0351544     -0.424904   0.150463   -0.455336 
 -0.300982   0.405588   -0.442091   …   0.272216   0.0822544  -0.0216966
 -0.462156  -0.337422   -0.151329      -0.399635   0.128237    0.160793 
 -0.436189   0.15099     0.263144       0.333617   0.230546    0.337798 